# 1. Data and python environment

## 1.1. Copernicus Marine Service Hindcast marine wave stoke velocity dataset

Using CMEMS access it was downloaded the dataset with forcing variables (Stokes velocity vector) for the East mediterranean se in the date 2021-07-23 to 2021-08-12, accordin to the study conducted about the oil spills that occurred at the Syrian Baniyas Station in the Eastern Mediterranean on August 23, 2021 (DOI: 10.1016/j.marpolbul.2023.115887).
The data was downloaded using the browser access to Copernicus Marine Services, product id: MEDSEA_MULTIYEAR_WAV_006_012 - [data.marine.copernicus.eu](https://data.marine.copernicus.eu/product/MEDSEA_MULTIYEAR_WAV_006_012/download?dataset=med-hcmr-wav-rean-h_202411).

file is stored locally in the same folder of this notebook [med-hcmr-wav-rean-h_cyprus-aug-2021.nc](med-hcmr-wav-rean-h_cyprus-aug-2021.nc)



In [2]:
# check dataset downloaded
import xarray as xr
DS = xr.open_dataset('med-hcmr-wav-rean-h_cyprus-aug-2021.nc')
DS

<xarray.Dataset> Size: 85MB
Dimensions:    (time: 505, latitude: 127, longitude: 111)
Coordinates:
  * time       (time) datetime64[ns] 4kB 2021-08-23T23:00:00 ... 2021-09-13T2...
  * latitude   (latitude) float32 508B 32.9 32.94 32.98 ... 38.06 38.1 38.15
  * longitude  (longitude) float32 444B 31.71 31.75 31.79 ... 36.21 36.25 36.29
Data variables:
    VHM0       (time, latitude, longitude) float32 28MB ...
    VSDX       (time, latitude, longitude) float32 28MB ...
    VSDY       (time, latitude, longitude) float32 28MB ...
Attributes: (12/13)
    Conventions:       CF-1.11
    title:             Wave fields (2D) - Hourly Instantaneous
    institution:       Hellenic Centre for Marine Research (HCMR)- Athens,Greece
    producer:          CMEMS-MED Monitoring and Forecasting Centre
    source:            MEDWAM3
    credit:            Copernicus Marine Environment Monitoring Service (CMEMS)
    ...                ...
    references:        Please check in CMEMS catalogue the INFO section for p...
    comment:           Please check in CMEMS catalogue the INFO section for p...
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    subset:productId:  MEDSEA_MULTIYEAR_WAV_006_012
    subset:datasetId:  med-hcmr-wav-rean-h_202411
    subset:date:       2025-07-03T07:30:49.259Z

## 1.2 Simulation data

To conduct the validation we use a multi-source release scenario.
It is a simulation of the condition of oil slik reported by SAR data acquired for the study conducted by R.M. Abou Samra and R.R. Ali (DOI: 10.1016/j.marpolbul.2023.115887), at the date of 2021-08-29 16:36.

To use `heco.run()` function in a multi-source condition we use a multitude of input file, with a variation of values only for source location.

Template yaml file is the following
```yaml
input:
  dataset_file_name: med-hcmr-wav-rean-h_cyprus-aug-2021.nc
  lat0: #variable
  lon0: #variable
  sim_diffusion_coeff: 10.0
  sim_duration_h: 72.0
  sim_particles: 100.0
  sim_timedelta_s: 3600.0
  spill_release_duration_h: 1.0
  time0: '2021-08-24 16:36:07'
  volume_spilled_m3: 100.0

```


In [3]:
# create an array of points coordinates
# import point list from geojson file

import json
with open('virtual_points_oil_spill/virtual_origin_oil_spill_points_2.geojson', 'r') as f:
    geojson_data = f.read()
    geojson_points = json.loads(geojson_data)

# create a template yaml file
template = """input:
  dataset_file_name: med-hcmr-wav-rean-h_cyprus-aug-2021.nc
  lat0: {lat}
  lon0: {lon}
  sim_diffusion_coeff: 10.0
  sim_duration_h: 72.0
  sim_particles: 100.0
  sim_timedelta_s: 3600.0
  spill_release_duration_h: 1.0
  time0: '2021-08-29 16:36:07'
  volume_spilled_m3: 100.0    
"""
# create a yaml file for each point
id = 0
for point in geojson_points['features']:
    lat = point['geometry']['coordinates'][1]
    lon = point['geometry']['coordinates'][0]
    yaml_content = template.format(lat=lat, lon=lon)

    # create an incremental id
    id += 1

    # write to a yaml file
    file_name = f"virtual_points_oil_spill/virtual_origin_oil_spill_{id}.yaml"
    with open(file_name, 'w') as yaml_file:
        yaml_file.write(yaml_content)
    
    print(f"Created {file_name} for point ({lat}, {lon})")


Created virtual_points_oil_spill/virtual_origin_oil_spill_1.yaml for point (35.35416718134785, 35.12500110973005)
Created virtual_points_oil_spill/virtual_origin_oil_spill_2.yaml for point (35.39583387828989, 35.25000112706958)
Created virtual_points_oil_spill/virtual_origin_oil_spill_3.yaml for point (35.35416718134785, 35.08333443728354)
Created virtual_points_oil_spill/virtual_origin_oil_spill_4.yaml for point (35.35416718134785, 35.16666778217656)
Created virtual_points_oil_spill/virtual_origin_oil_spill_5.yaml for point (35.687500756884205, 34.95833441994401)
Created virtual_points_oil_spill/virtual_origin_oil_spill_6.yaml for point (35.687500756884205, 34.9166677474975)
Created virtual_points_oil_spill/virtual_origin_oil_spill_7.yaml for point (35.604167363000116, 34.87500107505099)
Created virtual_points_oil_spill/virtual_origin_oil_spill_8.yaml for point (35.687500756884205, 35.00000109239052)
Created virtual_points_oil_spill/virtual_origin_oil_spill_9.yaml for point (35.604167

![img](virtual_points_oil_spill/figure7b-Abou_Samra&Ali_2024.png)

# 2. Run Simulation

Using a custom function we will run a Lagrangian model that simulate the emission of spill started from data retrieved by SAR in 2021-08-19

In [4]:
import heco
import geopandas as gpd
import glob

# create folder if not exists
import os
if not os.path.exists('results'):
    os.makedirs('results')

id = 0
for yaml_file in sorted(glob.glob('virtual_points_oil_spill/virtual_origin_oil_spill_*.yaml')):
    id += 1
    print(f"Running HECO for {yaml_file}")
    output = heco.run(yaml_file, 'VSDX', 'VSDY')
    # save result to csv
    output.to_csv(f"results/heco_results_{id}.csv", index=False)
    # create gejson file
    gdf = gpd.GeoDataFrame(output, geometry=gpd.points_from_xy(output.lon, output.lat))
    gdf.crs = "EPSG:4326"
    gdf.to_file(f"results/heco_results_{id}.geojson", driver='GeoJSON')



Running HECO for virtual_points_oil_spill/virtual_origin_oil_spill_1.yaml
Dataset med-hcmr-wav-rean-h_cyprus-aug-2021.nc opened
Volume per particle considered: 1.0 m3
discrete spill step 0 , release time 2021-08-29 16:36:07
Running HECO for virtual_points_oil_spill/virtual_origin_oil_spill_10.yaml
Dataset med-hcmr-wav-rean-h_cyprus-aug-2021.nc opened
Volume per particle considered: 1.0 m3
discrete spill step 0 , release time 2021-08-29 16:36:07
Running HECO for virtual_points_oil_spill/virtual_origin_oil_spill_11.yaml
Dataset med-hcmr-wav-rean-h_cyprus-aug-2021.nc opened
Volume per particle considered: 1.0 m3
discrete spill step 0 , release time 2021-08-29 16:36:07
Running HECO for virtual_points_oil_spill/virtual_origin_oil_spill_12.yaml
Dataset med-hcmr-wav-rean-h_cyprus-aug-2021.nc opened
Volume per particle considered: 1.0 m3
discrete spill step 0 , release time 2021-08-29 16:36:07
Running HECO for virtual_points_oil_spill/virtual_origin_oil_spill_13.yaml
Dataset med-hcmr-wav-rean-

## 3. Aggregate results

Now aggregate results tables in ones and unique geojson

In [5]:
# merge all csv file in one pandas dataframe
import pandas as pd
results_global = pd.DataFrame()

id = 0
for csv_file in sorted(glob.glob('results/heco_results_*.csv')):
    id += 1
    # open csv and append in 
    df = pd.read_csv(csv_file)
    # insert colum with id of virtual origin spill id
    df.insert(0, 'virtual_origin_id', id)
    results_global = pd.concat([results_global, df], ignore_index=True)

In [6]:
# now create a geojson

gdf = gpd.GeoDataFrame(results_global, geometry=gpd.points_from_xy(results_global.lon, results_global.lat))
gdf.crs = "EPSG:4326"
gdf.to_file('results/heco_results_global.geojson', driver='GeoJSON')

## 3.3 Webmap and convex polygons

Using a custom function called `create_webmap`it is possible to generate an html that contain a LeafLet webmap and various EMODnet WMS Layers pre-configured.

It is also possible to pass the parameter `savepolygons = True` to convert points geometries in a convex-hull polygon that contain all particle for each time instant. This is a more efficent way to display and share the dispersions result across GIS user and map visualizers. The file will be saved in the default name `heco-polygons.geojson`

In [8]:
# export webmap

heco.create_webmap(
    HECOpoint_output_gdf_path = 'results/heco_results_global.geojson',
    EMODnetLayers = True,
    settingsFile_path = 'heco-validation-test.yaml',
    output_path = 'heco_map_global.html',
    savepolygons = True
)

GO to [heco_map.html](heco_map.html) to show the output